In [22]:
from pandas import Series, DataFrame
import statsmodels.api as sm
import pandas as pd
import nltk
import operator
import re,string
from patsy import dmatrices
%pylab inline
from nltk.corpus import stopwords
from itertools import chain
from collections import Counter
import random
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


## Part A

### A1

In [23]:
train_data = pd.read_csv('Train_rev1.csv')

In [24]:
train_data.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk


In order to find the top 5 POS, we need to first tokenize the full description.<br> 
We'll do this by first taking a random sample size of 2500.

In [25]:
random.seed(99)
sample_2500 = random.sample(range(len(train_data)),2500)
sample_data = train_data['FullDescription'][sample_2500]
sample_data[:3]

105900    Passionate about making lives better, Bupa is ...
99812     Category Manager  Milton Keynes High profile r...
52448     The Company: Our client enjoys a high profile ...
Name: FullDescription, dtype: object

Then we conduct nltk tokenize to our sample. But there's still some steps before tokenize.

In [26]:
full_des = sample_data.apply(lambda x:re.sub(r"[^a-zA-Z0-9\s]", "", x.lower())).sum()
des_words = nltk.word_tokenize(full_des)
des_words = [word for word in des_words if word.isalpha()==True] #get rid of punctuation

Then, get the top 5 POS.

In [ ]:
pos = nltk.pos_tag(des_words)

In [ ]:
cnt = Counter(tag for word,tag in pos)
cnt.most_common()[:5]

Looks like 'NN': Singular Noun, 'JJ': Adjective, 'IN': Preposition or subordinating conjunction, 'NNS': Pural Noun and 'DT': Determiner are the 5 most common POS in the description.<br>
<br>
Let's do the process again and this time, exclude the stopwords.

In [ ]:
#This step takes a *really* long time to run since the perceptor has to load everytime.
stop = set(stopwords.words('english'))
filtered_stopwords = [word for word in des_words if word not in stop]
filtered_pos = nltk.pos_tag(filtered_stopwords)
cnt2 = Counter(tag for word,tag in filtered_pos)
cnt2.most_common()[:5]

After excluding the stopwords, Singular Noun, Adjective and Pural Noun are still in the top 5 list. Verb(gerund or present participle) and Verb(non-3rd person singular present) are added in the top 5 POS.

### A2

First, calculate the frequency of words and sort them.

In [ ]:
#Calculate frequency.
fdist = nltk.FreqDist(des_words)

## Testing Zipf's law for top 100

In [ ]:
#Plot the top 100 against Zipf's Law
plt.figure(figsize=(20,10))
fdist.plot(100, cumulative=False)

In [ ]:
#Sorted by frequency
sort_fdist = pd.DataFrame(sorted(fdist.items(), key=operator.itemgetter(1),reverse=True))
most_common_100 = sort_fdist[:100]
most_common_100.columns = ['word','frequency']
most_common_100['rank'] = most_common_100['frequency'].rank(method='min',ascending=False)
most_common_100['zipf_law'] = [most_common_100["frequency"].max()/r for r in most_common_100['rank']]
most_common_100[:10]

In [ ]:
fig = plt.figure()

x = [math.log(c) for c in most_common_100['rank'].values]
y1 = [math.log(c) for c in most_common_100['frequency']]
y2 = [math.log(c) for c in most_common_100['zipf_law']]

ax1 = plt.plot(x,y1,label='Actual')
ax2 = plt.plot(x,y2,label='Theorical')

xlabel("log(rank)")
ylabel("log(frequency)")
title('Top 100')
plt.legend()
plt.show()

The above results show that the top 100 most does follow the Zipf's law in generally.<br>
Now, let's test the Zipf's law with the entire sample (2500 data points).

In [ ]:
#from sklearn import datasets, linear_model
most_common_100['Y'] = [math.log(c) for c in most_common_100['frequency']]
x = most_common_100['frequency'] / (most_common_100['frequency'].max() * most_common_100.shape[0])
most_common_100['X'] = [math.log(c) for c in x]

In [ ]:
y, X = dmatrices('Y ~ 0 + X', data=most_common_100, return_type='dataframe')

In [ ]:
model = sm.OLS(y, X)       # Set up the model
#model = linear_model.LinearRegression()
result = model.fit()       # Fit model (find the intercept and slopes)
print(result.summary())

We can see that the coeff is -0.9927, which is close to -1. Hence we can assume that the top 100 words in job description follow Zipf's law

## Testing Zipf's law for entire sample

In [ ]:
sort_fdist.columns = ['word','frequency']
sort_fdist['rank'] = sort_fdist['frequency'].rank(method='min',ascending=False)
sort_fdist['zipf_law'] = [sort_fdist["frequency"].max()/r for r in sort_fdist['rank']]
sort_fdist.head()

In [ ]:
fig = plt.figure()

x = [math.log(c) for c in sort_fdist['rank'].values]
y1 = [math.log(c) for c in sort_fdist['frequency']]
y2 = [math.log(c) for c in sort_fdist['zipf_law']]

ax1 = plt.plot(x,y1,label='Actual')
ax2 = plt.plot(x,y2,label='Theorical')

xlabel("log(rank)")
ylabel("log(frequency)")
title('Entire sample')
plt.legend()
plt.show()

### Testing Zipf's law empirically

In [ ]:
import statsmodels.api as sm
from patsy import dmatrices

In [ ]:
#from sklearn import datasets, linear_model
sort_fdist['Y'] = [math.log(c) for c in sort_fdist['frequency']]
x = sort_fdist['frequency'] / (sort_fdist['frequency'].max() * sort_fdist.shape[0])
sort_fdist['X'] = [math.log(c) for c in x]

In [ ]:
y, X = dmatrices('Y ~ 0 + X', data=sort_fdist, return_type='dataframe')

In [ ]:
model = sm.OLS(y, X)       # Set up the model
#model = linear_model.LinearRegression()
result = model.fit()       # Fit model (find the intercept and slopes)
print(result.summary())

The coeff is negative, however closer to 0. Hence we can conclude that in the sample of 2500, the words occurring are not representative of any power law, in this case Zipf's law. This can be attributed to the fact that the words with lower ranks do not behave in this fashion. We will test for the same in the following pieces of code

## Testing Zipf's law for last 15000 words

In [ ]:
y, X = dmatrices('Y ~ 0 + X', data=sort_fdist.tail(15000), return_type='dataframe')
model = sm.OLS(y, X)       # Set up the model
#model = linear_model.LinearRegression()
result = model.fit()       # Fit model (find the intercept and slopes)
print(result.summary())

We can clearly see here, that these words do not follow the power law.

To summarize, as a whole the data doesn't support Zipf's Law. However, Zipf's Law will work up to a certain frequency. In this case, we tested it empirically for up to 100 words and observed that those 100 words belonged to a specific distribution of Zipf's Law. We also tested for the last 15000 words and observed that they don't belong to the Zipf's distribution and those points might be driving the sample to not follow the power law. 

### A3

In [ ]:
from nltk.corpus import wordnet
#create a function that would return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
         return wordnet.ADV
    else:
        # As default pos in lemmatization is Noun
        return wordnet.NOUN

In [ ]:
wnl = WordNetLemmatizer()
#create an empty list to store lemmatized words
des_lem = []

In [ ]:
def wn_pos(filtered_pos):
    for word,pos in filtered_pos:
        des_lem.append(wnl.lemmatize(word,get_wordnet_pos(pos)))
        #print pos
        #print get_wordnet_pos(pos)
    return des_lem

In [ ]:
# Get the 10 most common words
fdist_2 = nltk.FreqDist(wn_pos(filtered_pos))
fdist_2.most_common(10)

The top 10 most common words that appears in the job descriptions are shown above.

# PART B

### B1 & B2
#### Model with numeric columns only

In [ ]:
#Reading training data
data = pd.read_csv('Train_rev1.csv')

In [ ]:
data.dtypes

In [ ]:
data_s = data[['LocationNormalized','ContractType','ContractTime','Category','SalaryNormalized']]
print(data_s.shape)
data_s.head()

In [ ]:
#Checking NA in Contract Type
data_s.ContractType.value_counts(dropna=False)

Since ~73% of ContractType is missing, we will not be using this column for our classification. Replacing NaN with "Full Time" will bias the data

In [ ]:
data_s = data[['LocationNormalized','ContractTime','Category','SalaryNormalized']]
print(data_s.shape)
df = data_s.dropna()
df.shape

By dropping all rows with missing values, we lost about 65K rows (~26%). We will be using the clean dataset going forward

In [ ]:
p=np.percentile(df['SalaryNormalized'],75)
def target(t):
    if t>p:
        return 1
    else:
        return 0
    
df['target'] = df['SalaryNormalized'].map(target)

#### Get a list of cities with highest cost of living.

In [ ]:
#Get the top 10 highest CoL data from https://abcfinance.co.uk/blog/the-true-cost-of-living-in-uk-cities/
high_cost = ['London','Milton Keynes','Bath','Reading','Aberdeen','Cambridge','Oxford','Portsmouth','Edinburgh','York']

In [ ]:
def location_class(s):
    if s in high_cost:
        return 'high'
    else:
        return 'low'
df['location_class'] = df['LocationNormalized'].map(location_class)

Now, get the dummies for each variables.

In [ ]:
categorical_columns = ['ContractTime', 'Category','location_class']
data_dummies = pd.get_dummies(df[categorical_columns],
                            prefix=categorical_columns,
                            columns=categorical_columns)
dummy_column_names = data_dummies.columns.values

In [ ]:
df2 = pd.concat([df, data_dummies], axis=1)

Now, build Naive Bayes model.

Creating equation

In [ ]:
formula = 'target ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in dummy_column_names]))

In [ ]:
Y, X = dmatrices(formula, df2, return_type='dataframe')
y = Y['target'].values

Since there is no validation dataset, we will create a testing/training sample here

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
from sklearn import naive_bayes
model = naive_bayes.BernoulliNB()
model.fit(X_train, y_train)

In [ ]:
#Checking training accuracy
from sklearn import metrics
prediction_train = model.predict(X_test)
print(metrics.accuracy_score(y_test, prediction_train))
print("Test data confusion matrix")
confusion_matrix(y_test, prediction_train)

## The accuracy of using numerical variables are 76.06% on the test data

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, prediction_train))

#### Model with text columns only

Creating the dataset for classification

In [ ]:
data_s = data[['FullDescription','SalaryNormalized']]

Creating target variable

In [ ]:
p=np.percentile(data_s['SalaryNormalized'],75)
def target(t):
    if t>p:
        return 'high'
    else:
        return 'low'
    
data_s['target'] = data_s['SalaryNormalized'].map(target)

Taking a sample of 2500 rows

In [ ]:
random.seed(99)
#sample = random.sample(range(len(data_s)),0.7 * len(data_s))
sample = random.sample(range(len(data_s)),2500)
df = data_s.loc[sample,:]
print(df.shape)

In [ ]:
df.head()

Cleaning the job description by:
1. Removing punctuation
2. Getting rid of stop words
3. Removing Numbers
4. Stripping excess whitespace

In [ ]:
import re
from nltk.corpus import stopwords
#removing punctuation and numbers
df['job_des'] = df['FullDescription'].apply(lambda x:re.sub(r'[^a-zA-z\s]', ' ', x.lower()))

In [ ]:
#remove white spaces
df['job_des'] = df.job_des.apply(lambda x:re.sub(r'\s+', ' ', x))

In [ ]:
#remove stopwords
stop = set(stopwords.words('english'))
df['job_des_clean'] = df.job_des.apply(lambda x: [word for word in x.split() if word not in stop])
df.head()

Creating training data for Naive Bayes

Using just first 2000 words as features

In [ ]:
job_des_all = df['job_des_clean'].sum()

In [ ]:
all_words = nltk.FreqDist(job_des_all)
word_features = list(all_words)[:2000]
len(word_features)

In [ ]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [ ]:
df2 = df[['target','job_des_clean']]
t = list(zip(df2.melt('target').value,df2.melt('job_des_clean').value))

In [ ]:
featuresets = [(document_features(x[0]), x[1]) for x in t]

In [ ]:
train_set, test_set = featuresets[2000:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
print(nltk.classify.accuracy(classifier, test_set))

## The accuracy of the text only model is 79.6%. We used Binomial Naive Bayes classification for both text and numeric models.

## The top 10 words indicating high salary and low salary.

In [ ]:
classifier.show_most_informative_features(10)

## B3

We used Bag of words to convert the text data to numeric data. 

In [ ]:
#import zipfile
import pandas as pd
import numpy as np
import nltk as nl
import os
pd.set_option('display.max_columns', None)

In [ ]:
# dataset_name = 'job_salary'
# zf = zipfile.ZipFile('../data/'+dataset_name+ '.zip')
# files = zf.infolist()

# for f in files:
#     print("file present here is", f.filename)
#     df = pd.read_csv(zf.open(f.filename))
#if the data was in zipfiles, we would read it with the code above^
df = pd.read_csv('Train_rev1.csv')

In [ ]:
df.shape

In [ ]:
df_chosen = df.sample(2500, random_state=99)

In [ ]:
col_names = ['location','contract_type','contract_time','type_of_job','salary']
num_data = df_chosen[['LocationNormalized','ContractType','ContractTime','Category','SalaryNormalized']]
num_data.columns = col_names 

In [ ]:
p=np.percentile(num_data['salary'],75)
def target(t):
    if t>p:
        return 1
    else:
        return 0
    
num_data['target'] = num_data['salary'].map(target)

In [ ]:
high_cost = ['London','Milton Keynes','Bath','Reading','Aberdeen','Cambridge','Oxford','Portsmouth','Edinburgh','York']
def location_class(s):
    if s in high_cost:
        return 1
    else:
        return 0
num_data['location_class'] = num_data['location'].map(location_class)

In [ ]:
num_data.pop('location')
num_data.head()

In [ ]:
num_data[['contract_type', 'contract_time','type_of_job']].describe()

### Since, many of the observations in 'contract_time', 'contract_type' are null, let's impute them using the most frequent value for  them

In [ ]:
num_data['contract_time'] = num_data['contract_time'].fillna('permanent')
num_data['contract_type'] = num_data['contract_type'].fillna('full_time')
num_data[['contract_type', 'contract_time','type_of_job']].isnull().sum()

In [ ]:
df_model = num_data[['contract_type', 'contract_time', 'type_of_job','location_class','target']]
categorical_columns = ['contract_type', 'contract_time', 'type_of_job']
#num_data['contract_type'] = num_data['contract_time'].astype('category')
df_model = pd.get_dummies(df_model, columns= categorical_columns)

In [ ]:
df_model.shape   ## Numerical data tranformed to dummy variables

### Now we include Text data, and make a boolean vector for the corpus' vocabulary
### Using a Bag o' Words approach to transform text to numerics

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = list(df_chosen['FullDescription'])
vectorizer = CountVectorizer()
corpus_bool = vectorizer.fit_transform(corpus).todense() 
corpus_bool[:10]

In [ ]:
corpus_bool.shape

In [ ]:
word_df = pd.DataFrame(vectorizer.fit_transform(corpus).todense(), columns= vectorizer.vocabulary_ )
dict_ = vectorizer.vocabulary_
dict_sorted = sorted(dict_ , key= lambda x: dict_[x], reverse=True) ## Sorting the dictionary based on frequency of words
dict_sorted = dict_sorted[:3000] ## Retaining only 3000 words
dictfilt = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y) ])
dict_filtrered = dictfilt(dict_, dict_sorted)

In [ ]:
keys = list(dictfilt(dict_, dict_sorted).keys())
word_df = word_df[keys]

In [ ]:
print(word_df.shape) 
#word_df.head()

In [ ]:
df_model.rename(columns={'target':'my_response_variable'}, inplace=True)
df_model = df_model.reset_index(drop=True)
word_df= word_df.reset_index(drop=True)

In [ ]:
full_data = df_model.join(word_df)
full_data.shape

## Running Classification Task

In [ ]:
Y_target = full_data['my_response_variable']
full_data.drop('my_response_variable', inplace=True, axis=1)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(full_data, Y_target, test_size=500, random_state=42)

In [ ]:
## User Defined Function for model training
def fit_the_model(X_train, y_train, model_name):
    if(model_name == 'NB'):
        clf = BernoulliNB()
        clf.fit(X_train, y_train)
        print(" Model fitting done by Naive Bayes Bernoulli")
        
    elif(model_name == 'RF'):
        clf = RandomForestClassifier()
        clf.fit(X_train, y_train)
        print("Model fitting done by Random Forest")
        
    return clf
## User Defined Function for model evaluation

def classification_model_evaluation(model, X_train, y_train, X_test, y_test):
    prediction_train = model.predict(X_train)
    prediction_test = model.predict(X_test)
    print ("Accuracy on Training data is", metrics.accuracy_score(y_train, prediction_train))
    print ("Accuracy on Test data is", metrics.accuracy_score(y_test, prediction_test))
    print('\n')
    print("Confusion Matrix FOR TEST Obtained is: \n ", confusion_matrix(y_test, prediction_test))
    print('\n')
    print("Confusion Matrix FOR TRAIN Obtained is: \n ", confusion_matrix(y_train, prediction_train))
    report = classification_report(y_test, prediction_test)
    print('\n')
    print("Classification Report \n", report)
    return ("Printed All the metrics for your classification model")

In [ ]:
my_mod = fit_the_model(X_train, y_train, 'NB')
classification_model_evaluation(my_mod,X_train, y_train, X_test, y_test )

In [ ]:
my_rf_mod = fit_the_model(X_train, y_train, 'RF')
classification_model_evaluation(my_rf_mod,X_train, y_train, X_test, y_test )

### It is evident that Naive Bayes performs better , as it doesn't result in overfittng, unlike Random Forest

## Which model – numeric only, text only and hybrid – provided the highest accuracy in predicting high/low salary? Did the result surprise you? Why or why not?

| Model               | Test Accuracy     |
|---------------      |---------------    |
| Numeric only        |      76%          |
| Text only           |     79.6%         |
| Hybrid Naive Bayes  |     77.6%         |
| Hybrid Random Forest|     76.8%         |


The results did surprise us, because we assumed that the hybrid, which is made up of numbers and text, should be more accurate because of more data. However, the text only model was the most accurate. 